Pydantic validation of LLM response to make sure its in json format with right data types.

In [1]:
!pip install --upgrade fireworks-ai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 577.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [2]:
!pip install pydantic==2.5.0
!pip install pydantic-core==2.14.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires

In [3]:

import fireworks.client



In [4]:
fireworks.client.api_key = "W4etuSqWlXPUIa8gsuz8g9kCvz6KGNsSF4q8FePNjpBLOgfF"

Web scrapping https://en.wikipedia.org/wiki/List_of_2023_box_office_number-one_films_in_the_United_States .
I have only extracted data from the table



In [5]:
import requests
from bs4 import BeautifulSoup

def scrape_box_office(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'wikitable'})
        box_office_data = []
        for row in table.find_all('tr')[1:]:
            columns = row.find_all(['th', 'td'])
            data = [column.get_text(strip=True) for column in columns]
            box_office_data.append(data)

        return box_office_data
    else:
        print(f"Error: Unable to fetch data. Status Code: {response.status_code}")

url = 'https://en.wikipedia.org/wiki/List_of_2023_box_office_number-one_films_in_the_United_States'

result = scrape_box_office(url)
string = ""
for row in result:
    for i in range(len(row)):
      if i==0:
        pass
      else:
        string += row[i] + " "
    string += '\n'


print(string)

January 8, 2023 Avatar: The Way of Water $45,838,986 Black Panther: Wakanda ForeverandAvatar: The Way of Waterbecame the first two films to consecutively top the box office for four consecutive weekends each sinceThe Hunger Games: Mockingjay – Part 2andStar Wars: The Force Awakensin 2015 and 2016. [2] 
January 15, 2023 $32,824,684 Black Panther: Wakanda ForeverandAvatar: The Way of Waterbecame the first two films to consecutively top the box office for five consecutive weekends each sinceStakeoutandFatal Attractionin 1987. [3] 
January 22, 2023 $20,133,106 Avatar: The Way of Waterbecame the first film sinceAvatarto top the box office for six consecutive weekends. It also became the first film sinceSpider-Man: No Way Hometo top the box office in its sixth week of release, as well as for six total weekends. During the weekend, the film surpassed $2 billion at the global box office. [4] 
January 29, 2023 $15,968,532 Avatar: The Way of Waterbecame the first film sinceAvatarto top the box o

Specifying a pydantic schema

In [6]:
from pydantic import BaseModel, ValidationError
import json
class WebScrape1(BaseModel):
    date: str
    film: str
    gross: int

In [7]:
response_schema_dict = WebScrape1.model_json_schema()
response_schema_json = json.dumps(response_schema_dict)
response_schema_json

'{"properties": {"date": {"title": "Date", "type": "string"}, "film": {"title": "Film", "type": "string"}, "gross": {"title": "Gross", "type": "integer"}}, "required": ["date", "film", "gross"], "title": "WebScrape1", "type": "object"}'

Creating a prompt

In [8]:
# question = "1 January 8, 2023 Avatar: The Way of Water $45,838,986 Black Panther: Wakanda ForeverandAvatar: The Way of Waterbecame the first two films to consecutively top the box office for four consecutive weekends each sinceThe Hunger Games: Mockingjay – Part 2andStar Wars: The Force Awakensin 2015 and 2016. [2] "
prompt = f"""I will provide you with a unstructured data and you have to convert it into the following format and Do not include the provided properties json in the output:
```json
{response_schema_dict}
```
Make sure you only have the required json as output and no other text.

Data: {string}

only generate extracted data json.
The provided format should be strictly followed.

"""

In [9]:
prompt

"I will provide you with a unstructured data and you have to convert it into the following format and Do not include the provided properties json in the output:\n```json\n{'properties': {'date': {'title': 'Date', 'type': 'string'}, 'film': {'title': 'Film', 'type': 'string'}, 'gross': {'title': 'Gross', 'type': 'integer'}}, 'required': ['date', 'film', 'gross'], 'title': 'WebScrape1', 'type': 'object'}\n```\nMake sure you only have the required json as output and no other text.\n\nData: January 8, 2023 Avatar: The Way of Water $45,838,986 Black Panther: Wakanda ForeverandAvatar: The Way of Waterbecame the first two films to consecutively top the box office for four consecutive weekends each sinceThe Hunger Games: Mockingjay – Part 2andStar Wars: The Force Awakensin 2015 and 2016. [2] \nJanuary 15, 2023 $32,824,684 Black Panther: Wakanda ForeverandAvatar: The Way of Waterbecame the first two films to consecutively top the box office for five consecutive weekends each sinceStakeoutandFat

Creating functions for text generation and error handling

In [10]:
def askLLM(prompt):
    completion = fireworks.client.ChatCompletion.create(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    messages=[
      {
        "role": "user",
        "content": prompt,
      }
    ],
    n=1,
    max_tokens=4000,
    temperature=0.1,
    top_p=0.9,
   )
    return completion.choices[0].message.content


def handle_validation_failure(error, prompt, num_tries):
  if num_tries > 3:
    return "Couldnt validate LLM response"
  else:
    out = askLLM(error + prompt+ "Make sure the output is json")
    print(out)
    return out


Providing initial prompt and viewing the results

In [11]:
completion = askLLM(prompt)

In [12]:
completion

'[\n{"date": "January 8, 2023", "film": "Avatar: The Way of Water", "gross": 45838986},\n{"date": "January 15, 2023", "film": "Black Panther: Wakanda Forever", "gross": 32824684},\n{"date": "January 22, 2023", "film": "Avatar: The Way of Water", "gross": 20133106},\n{"date": "January 29, 2023", "film": "Avatar: The Way of Water", "gross": 15968532},\n{"date": "February 5, 2023", "film": "Knock at the Cabin", "gross": 14127170},\n{"date": "February 12, 2023", "film": "Magic Mike\'s Last Dance", "gross": 8305317},\n{"date": "February 19, 2023", "film": "Ant-Man and the Wasp: Quantumania", "gross": 106109650},\n{"date": "February 26, 2023", "film": null, "gross": 31964803},\n{"date": "March 5, 2023", "film": "Creed III", "gross": 58370007},\n{"date": "March 12, 2023", "film": "Scream VI", "gross": 44447270},\n{"date": "March 19, 2023", "film": "Shazam! Fury of the Gods", "gross": 30111158},\n{"date": "March 26, 2023", "film": "John Wick: Chapter 4", "gross": 73817950},\n{"date": "April 2,

In [13]:
max_tries = 0

In [14]:
while(max_tries < 3):
  try:
    json_objects = json.loads(completion)
    break
  except:
    completion = handle_validation_failure("Output Not json", prompt, max_tries)
    max_tries+=1


Pydantic Validations for each individual json extraction. This approach is used in order to avoid regeneration everytime there is an error in a single json. Errors are extracted and displayed to the user

In [15]:
failed = []
validated = []
for i in range(len(json_objects)):
  try:
      validated_data = WebScrape1.parse_obj(json_objects[i])
      print("Validation successful!")
      validated.append(json_objects[i])
      print("Validated data:", validated_data.dict())
  except ValidationError as e:
      print("Validation failed!")
      failed.append(json_objects[i])
      print(e.json())

print(failed)

Validation successful!
Validated data: {'date': 'January 8, 2023', 'film': 'Avatar: The Way of Water', 'gross': 45838986}
Validation successful!
Validated data: {'date': 'January 15, 2023', 'film': 'Black Panther: Wakanda Forever', 'gross': 32824684}
Validation successful!
Validated data: {'date': 'January 22, 2023', 'film': 'Avatar: The Way of Water', 'gross': 20133106}
Validation successful!
Validated data: {'date': 'January 29, 2023', 'film': 'Avatar: The Way of Water', 'gross': 15968532}
Validation successful!
Validated data: {'date': 'February 5, 2023', 'film': 'Knock at the Cabin', 'gross': 14127170}
Validation successful!
Validated data: {'date': 'February 12, 2023', 'film': "Magic Mike's Last Dance", 'gross': 8305317}
Validation successful!
Validated data: {'date': 'February 19, 2023', 'film': 'Ant-Man and the Wasp: Quantumania', 'gross': 106109650}
Validation failed!
[{"type":"string_type","loc":["film"],"msg":"Input should be a valid string","input":null,"url":"https://errors

<ipython-input-15-63b98083a3f4>:5: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  validated_data = WebScrape1.parse_obj(json_objects[i])
<ipython-input-15-63b98083a3f4>:8: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  print("Validated data:", validated_data.dict())


Displaying the failed prompts

In [16]:
for i in failed:
  print(i)

{'date': 'February 26, 2023', 'film': None, 'gross': 31964803}
{'date': 'May 14, 2023', 'film': None, 'gross': 62008548}
{'date': 'August 6, 2023', 'film': None, 'gross': 53008647}
{'date': 'September 17, 2023', 'film': None, 'gross': 8550110}


In [17]:
for i in validated:
  print(i)

{'date': 'January 8, 2023', 'film': 'Avatar: The Way of Water', 'gross': 45838986}
{'date': 'January 15, 2023', 'film': 'Black Panther: Wakanda Forever', 'gross': 32824684}
{'date': 'January 22, 2023', 'film': 'Avatar: The Way of Water', 'gross': 20133106}
{'date': 'January 29, 2023', 'film': 'Avatar: The Way of Water', 'gross': 15968532}
{'date': 'February 5, 2023', 'film': 'Knock at the Cabin', 'gross': 14127170}
{'date': 'February 12, 2023', 'film': "Magic Mike's Last Dance", 'gross': 8305317}
{'date': 'February 19, 2023', 'film': 'Ant-Man and the Wasp: Quantumania', 'gross': 106109650}
{'date': 'March 5, 2023', 'film': 'Creed III', 'gross': 58370007}
{'date': 'March 12, 2023', 'film': 'Scream VI', 'gross': 44447270}
{'date': 'March 19, 2023', 'film': 'Shazam! Fury of the Gods', 'gross': 30111158}
{'date': 'March 26, 2023', 'film': 'John Wick: Chapter 4', 'gross': 73817950}
{'date': 'April 2, 2023', 'film': 'Dungeons & Dragons: Honor Among Thieves', 'gross': 37205784}
{'date': 'Apri

Sorting by date

In [18]:
prompt = f"""I will provide you with a unstructured data and you have to convert it into the following format and Do not include the provided properties json in the output:
```json
{response_schema_dict}
```
Make sure you only have the required json as output and no other text.

Data: {string}

only generate extracted data json. Sort the data according to the gross revenue.
The provided format should be strictly followed.

"""

In [19]:
completion_new = askLLM(prompt)

In [20]:
max_tries = 0

In [21]:

while(max_tries < 3):
  try:
    json_objects1 = json.loads(completion_new)
    break
  except:
    completion_new = handle_validation_failure("Output Not json, do not produce schema in the response again", prompt, max_tries)
    max_tries+=1
    print(f"Try {max_tries}")

In [22]:
completion_new

'[\n{"date": "January 8, 2023", "film": "Avatar: The Way of Water", "gross": 45838986},\n{"date": "January 15, 2023", "film": "Black Panther: Wakanda Forever", "gross": 32824684},\n{"date": "January 15, 2023", "film": "Avatar: The Way of Water", "gross": 32824684},\n{"date": "January 22, 2023", "film": "Avatar: The Way of Water", "gross": 20133106},\n{"date": "January 29, 2023", "film": "Avatar: The Way of Water", "gross": 15968532},\n{"date": "February 5, 2023", "film": "Knock at the Cabin", "gross": 14127170},\n{"date": "February 12, 2023", "film": "Magic Mike\'s Last Dance", "gross": 8305317},\n{"date": "February 19, 2023", "film": "Ant-Man and the Wasp: Quantumania", "gross": 106109650},\n{"date": "February 26, 2023", "gross": 31964803},\n{"date": "March 5, 2023", "film": "Creed III", "gross": 58370007},\n{"date": "March 12, 2023", "film": "Scream VI", "gross": 44447270},\n{"date": "March 19, 2023", "film": "Shazam! Fury of the Gods", "gross": 30111158},\n{"date": "March 26, 2023",

In [23]:
json_objects1

[{'date': 'January 8, 2023',
  'film': 'Avatar: The Way of Water',
  'gross': 45838986},
 {'date': 'January 15, 2023',
  'film': 'Black Panther: Wakanda Forever',
  'gross': 32824684},
 {'date': 'January 15, 2023',
  'film': 'Avatar: The Way of Water',
  'gross': 32824684},
 {'date': 'January 22, 2023',
  'film': 'Avatar: The Way of Water',
  'gross': 20133106},
 {'date': 'January 29, 2023',
  'film': 'Avatar: The Way of Water',
  'gross': 15968532},
 {'date': 'February 5, 2023', 'film': 'Knock at the Cabin', 'gross': 14127170},
 {'date': 'February 12, 2023',
  'film': "Magic Mike's Last Dance",
  'gross': 8305317},
 {'date': 'February 19, 2023',
  'film': 'Ant-Man and the Wasp: Quantumania',
  'gross': 106109650},
 {'date': 'February 26, 2023', 'gross': 31964803},
 {'date': 'March 5, 2023', 'film': 'Creed III', 'gross': 58370007},
 {'date': 'March 12, 2023', 'film': 'Scream VI', 'gross': 44447270},
 {'date': 'March 19, 2023',
  'film': 'Shazam! Fury of the Gods',
  'gross': 30111158},

In [24]:
failed = []
validated = []
for i in range(len(json_objects1)):
  try:
      validated_data = WebScrape1.parse_obj(json_objects1[i])
      print("Validation successful!")
      validated.append(json_objects1[i])
      print("Validated data:", validated_data.dict())
  except ValidationError as e:
      print("Validation failed!")
      failed.append(json_objects1[i])
      print(e.json())

print(failed)

Validation successful!
Validated data: {'date': 'January 8, 2023', 'film': 'Avatar: The Way of Water', 'gross': 45838986}
Validation successful!
Validated data: {'date': 'January 15, 2023', 'film': 'Black Panther: Wakanda Forever', 'gross': 32824684}
Validation successful!
Validated data: {'date': 'January 15, 2023', 'film': 'Avatar: The Way of Water', 'gross': 32824684}
Validation successful!
Validated data: {'date': 'January 22, 2023', 'film': 'Avatar: The Way of Water', 'gross': 20133106}
Validation successful!
Validated data: {'date': 'January 29, 2023', 'film': 'Avatar: The Way of Water', 'gross': 15968532}
Validation successful!
Validated data: {'date': 'February 5, 2023', 'film': 'Knock at the Cabin', 'gross': 14127170}
Validation successful!
Validated data: {'date': 'February 12, 2023', 'film': "Magic Mike's Last Dance", 'gross': 8305317}
Validation successful!
Validated data: {'date': 'February 19, 2023', 'film': 'Ant-Man and the Wasp: Quantumania', 'gross': 106109650}
Validat

<ipython-input-24-97e9214df6a9>:5: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  validated_data = WebScrape1.parse_obj(json_objects1[i])
<ipython-input-24-97e9214df6a9>:8: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  print("Validated data:", validated_data.dict())


In [25]:
for i in validated:
  print(i)

{'date': 'January 8, 2023', 'film': 'Avatar: The Way of Water', 'gross': 45838986}
{'date': 'January 15, 2023', 'film': 'Black Panther: Wakanda Forever', 'gross': 32824684}
{'date': 'January 15, 2023', 'film': 'Avatar: The Way of Water', 'gross': 32824684}
{'date': 'January 22, 2023', 'film': 'Avatar: The Way of Water', 'gross': 20133106}
{'date': 'January 29, 2023', 'film': 'Avatar: The Way of Water', 'gross': 15968532}
{'date': 'February 5, 2023', 'film': 'Knock at the Cabin', 'gross': 14127170}
{'date': 'February 12, 2023', 'film': "Magic Mike's Last Dance", 'gross': 8305317}
{'date': 'February 19, 2023', 'film': 'Ant-Man and the Wasp: Quantumania', 'gross': 106109650}
{'date': 'March 5, 2023', 'film': 'Creed III', 'gross': 58370007}
{'date': 'March 12, 2023', 'film': 'Scream VI', 'gross': 44447270}
{'date': 'March 19, 2023', 'film': 'Shazam! Fury of the Gods', 'gross': 30111158}
{'date': 'March 26, 2023', 'film': 'John Wick: Chapter 4', 'gross': 73817950}
{'date': 'April 2, 2023', 